# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [3]:
from keras.datasets import imdb

Using TensorFlow backend.


In [0]:
import sys
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
import itertools
import random

from sklearn.utils import shuffle

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [0]:
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [6]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
project_path = "/content/drive/My Drive/8march/"

In [0]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size


In [9]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

17473536/17464789 [==============================] - 1s 0us/step


In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [0]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [12]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    1,   14,   22,   16,   43,  530,
        973, 1622, 1385,   65,  458, 4468,   66, 3941,    4,  173,   36,
        256,    5,   25,  100,   43,  838,  112,   50,  670,    2,    9,
         35,  480,  284,    5,  150,    4,  172,  112,  167,    2,  336,
        385,   39,    4,  172, 4536, 1111,   17,  546,   38,   13,  447,
          4,  192,   50,   16,    6,  147, 2025,   19,   14,   22,    4,
       1920, 4613,  469,    4,   22,   71,   87,   

In [13]:
x_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [14]:
print('x_train shape', x_train.shape, '\nx_test shape', x_test.shape, '\ny_train shape', y_train.shape, '\ny_test shape', y_test.shape)

('x_train shape', (25000, 300), '\nx_test shape', (25000, 300), '\ny_train shape', (25000,), '\ny_test shape', (25000,))


**Get the word index and then Create key value pair for word and word_id**

In [15]:
INDEX_FROM=3   # word index offset

# Word to ID
word_to_id = imdb.get_word_index()   # Retrieves the dictionary mapping word indices back to words.
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNKOWN>"] = 2

# ID to word
id_to_word = {value:key for key,value in word_to_id.items()}

1654784/1641221 [==============================] - 0s 0us/step


In [16]:
word_to_id

{u'fawn': 34704,
 u'tsukino': 52009,
 u'nunnery': 52010,
 u'sonja': 16819,
 u'vani': 63954,
 u'woods': 1411,
 u'spiders': 16118,
 u'hanging': 2348,
 u'woody': 2292,
 u'trawling': 52011,
 u"hold's": 52012,
 u'comically': 11310,
 u'localized': 40833,
 u'disobeying': 30571,
 u"'royale": 52013,
 u"harpo's": 40834,
 u'canet': 52014,
 u'aileen': 19316,
 u'acurately': 52015,
 u"diplomat's": 52016,
 u'rickman': 25245,
 u'rumbustious': 52017,
 u'familiarness': 52018,
 u"spider'": 52019,
 u'hahahah': 68807,
 u"wood'": 52020,
 u'transvestism': 40836,
 u"hangin'": 34705,
 u'bringing': 2341,
 u'seamier': 40837,
 u'wooded': 34706,
 u'kid\xf4': 83924,
 u'grueling': 16820,
 u'wooden': 1639,
 u'wednesday': 16821,
 u"'prix": 52022,
 u'altagracia': 34707,
 u'circuitry': 52023,
 u'crotch': 11588,
 u'busybody': 57769,
 u"tart'n'tangy": 52024,
 u'burgade': 14132,
 u'thrace': 52026,
 u"tom's": 11041,
 u'snuggles': 52028,
 u'francesco': 29117,
 u'complainers': 52030,
 u'templarios': 52128,
 u'272': 40838,
 u'

In [17]:
id_to_word

{0: '<PAD>',
 1: '<START>',
 2: '<UNKOWN>',
 4: u'the',
 5: u'and',
 6: u'a',
 7: u'of',
 8: u'to',
 9: u'is',
 10: u'br',
 11: u'in',
 12: u'it',
 13: u'i',
 14: u'this',
 15: u'that',
 16: u'was',
 17: u'as',
 18: u'for',
 19: u'with',
 20: u'movie',
 21: u'but',
 22: u'film',
 23: u'on',
 24: u'not',
 25: u'you',
 26: u'are',
 27: u'his',
 28: u'have',
 29: u'he',
 30: u'be',
 31: u'one',
 32: u'all',
 33: u'at',
 34: u'by',
 35: u'an',
 36: u'they',
 37: u'who',
 38: u'so',
 39: u'from',
 40: u'like',
 41: u'her',
 42: u'or',
 43: u'just',
 44: u'about',
 45: u"it's",
 46: u'out',
 47: u'has',
 48: u'if',
 49: u'some',
 50: u'there',
 51: u'what',
 52: u'good',
 53: u'more',
 54: u'when',
 55: u'very',
 56: u'up',
 57: u'no',
 58: u'time',
 59: u'she',
 60: u'even',
 61: u'my',
 62: u'would',
 63: u'which',
 64: u'only',
 65: u'story',
 66: u'really',
 67: u'see',
 68: u'their',
 69: u'had',
 70: u'can',
 71: u'were',
 72: u'me',
 73: u'well',
 74: u'than',
 75: u'we',
 76: u'much'

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [18]:
import keras

#Clear out tensorflow memory
keras.backend.clear_session()

# Define Model
model = keras.Sequential()

# Embedding layer
model.add(keras.layers.Embedding(vocab_size, 16))

# Pooling layer
model.add(keras.layers.GlobalAveragePooling1D())

# 1st dense layer
model.add(keras.layers.Dense(64, activation='relu'))

# Dropout layer
model.add(keras.layers.Dropout(0.5))

# 2nd dense layer
model.add(keras.layers.Dense(32, activation="relu"))

# 3rd dense layer
model.add(keras.layers.Dense(16, activation="relu"))

# Output layer
model.add(keras.layers.Dense(1, activation="sigmoid"))

# Summary of the model
model.summary()

W0308 17:34:36.777930 140444678768512 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0308 17:34:36.779367 140444678768512 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0308 17:34:36.799038 140444678768512 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0308 17:34:36.800904 140444678768512 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0308 17:34:36.803313 14

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1088      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total para

In [20]:
# Loss and Optimizer
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

# Training the model
model.fit(x_train, y_train, epochs=100, batch_size=5000, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 0s 20us/step - loss: 0.1023 - acc: 0.9681 - val_loss: 0.4352 - val_acc: 0.8600
Epoch 2/100
25000/25000 [==============================] - 0s 5us/step - loss: 0.0885 - acc: 0.9761 - val_loss: 0.4423 - val_acc: 0.8647
Epoch 3/100
25000/25000 [==============================] - 0s 6us/step - loss: 0.0822 - acc: 0.9792 - val_loss: 0.4372 - val_acc: 0.8630
Epoch 4/100
25000/25000 [==============================] - 0s 6us/step - loss: 0.0806 - acc: 0.9816 - val_loss: 0.4480 - val_acc: 0.8651
Epoch 5/100
25000/25000 [==============================] - 0s 6us/step - loss: 0.0782 - acc: 0.9815 - val_loss: 0.4444 - val_acc: 0.8630
Epoch 6/100
25000/25000 [==============================] - 0s 5us/step - loss: 0.0761 - acc: 0.9821 - val_loss: 0.4547 - val_acc: 0.8643
Epoch 7/100
25000/25000 [==============================] - 0s 6us/step - loss: 0.0733 - acc: 0.9842 - val_loss: 0.4567 - val_ac

In [0]:
#Report the Accuracy of the model

In [0]:
# Prediction
y_pred = model.predict(x_test)

In [22]:
from sklearn.metrics import accuracy_score

AccuracyScore_train = model.evaluate(x_train, y_train)
print("Accuracy Score of train data:", round(AccuracyScore_train[1]*100,2), "%\n\n")

AccuracyScore_test = model.evaluate(x_test, y_test)
print("Accuracy Score of test data:", round(AccuracyScore_test[1]*100,2), "%")

25000/25000 [==============================] - 1s 43us/step
('Accuracy Score of train data:', 99.86, '%\n\n')
25000/25000 [==============================] - 1s 41us/step
('Accuracy Score of test data:', 84.3, '%')


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [23]:
import numpy as np

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [keras.backend.function([inp, keras.backend.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
test = np.random.random(vocab_size)[np.newaxis,...]
layer_outs = [func([test, 1.]) for func in functors]
print(layer_outs)

[[array([[[ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812],
        [ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812],
        [ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812],
        ...,
        [ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812],
        [ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812],
        [ 0.01711977,  0.01019934,  0.01974214, ..., -0.02063373,
          0.01369919, -0.00132812]]], dtype=float32)], [array([[ 0.01711977,  0.01019933,  0.01974214,  0.00884489, -0.01037353,
         0.00777128,  0.00560889, -0.02210593, -0.01104617,  0.01058729,
        -0.00590373,  0.00443505,  0.02176663, -0.02063374,  0.0136992 ,
        -0.00132812]], dtype=float32)], [array([[0.00170159, 0.00672223, 0.00533827, 0.0058391 , 0.00576651,
        0.        , 0.002